In [119]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from io import StringIO
import os

In [120]:
url = "https://pt.wikipedia.org/wiki/Lista_de_presidentes_da_C%C3%A2mara_dos_Deputados_do_Brasil"

In [121]:
response = requests.get(url)
html_content = response.text
soup = BeautifulSoup(html_content, 'html.parser')
tables = soup.find_all('table', {'class': 'wikitable'})


In [122]:
table = tables[1]
dfs = pd.read_html(StringIO(str(table)))
df = dfs[0]
df.head()

Nº  \
                                     Primeira República Brasileira (1889-1930)   
  Poder Legislativo fechado entre 15 de novembro de 1889 e 1° de junho de 1891   
0                                                 58                             
1                                                 59                             
2                                                 60                             
3                                                 61                             
4                                                 62                             

                                                              Nome do deputado  \
                                     Primeira República Brasileira (1889-1930)   
  Poder Legislativo fechado entre 15 de novembro de 1889 e 1° de junho de 1891   
0                              João da Matta Machado                             
1                   Bernardino José de Campos Júnior                             
2                          João Lopes Ferreira Filho                             
3                    Francisco de Assis Rosa e Silva                             
4                                   Artur César Rios                             

                                                                        Imagem  \
                                     Primeira República Brasileira (1889-1930)   
  Poder Legislativo fechado entre 15 de novembro de 1889 e 1° de junho de 1891   
0                                                NaN                             
1                                                NaN                             
2                                                NaN                             
3                                                NaN                             
4                                                NaN                             

                                                             Início do mandato  \
                                     Primeira República Brasileira (1889-1930)   
  Poder Legislativo fechado entre 15 de novembro de 1889 e 1° de junho de 1891   
0                                18 de junho de 1891                             
1                              31 de outubro de 1891                             
2                               18 de agosto de 1892                             
3                                 17 de maio de 1894                             
4                                 19 de maio de 1896                             

                                                                Fim do mandato  \
                                     Primeira República Brasileira (1889-1930)   
  Poder Legislativo fechado entre 15 de novembro de 1889 e 1° de junho de 1891   
0                              31 de outubro de 1891                             
1                               18 de agosto de 1892                             
2                                 17 de maio de 1894                             
3                                 19 de maio de 1896                             
4                                  9 de maio de 1899                             

                                                   Estado pelo qual foi Eleito  \
                                     Primeira República Brasileira (1889-1930)   
  Poder Legislativo fechado entre 15 de novembro de 1889 e 1° de junho de 1891   
0                                       Minas Gerais                             
1                                          São Paulo                             
2                                              Ceará                             
3                                         Pernambuco                             
4                                              Bahia                             

                                                                       Partido  \
                                     Primeira Repúbl

In [123]:
df.columns

MultiIndex([(                         'Nº', ...),
            (           'Nome do deputado', ...),
            (                     'Imagem', ...),
            (          'Início do mandato', ...),
            (             'Fim do mandato', ...),
            ('Estado pelo qual foi Eleito', ...),
            (                    'Partido', ...),
            (                    'Partido', ...),
            (                   'Eleições', ...),
            (                   'Eleições', ...)],
           )

In [124]:
df.columns = [' '.join(col).strip() for col in df.columns.values]

df.columns


Index(['Nº Primeira República Brasileira (1889-1930) Poder Legislativo fechado entre 15 de novembro de 1889 e 1° de junho de 1891',
       'Nome do deputado Primeira República Brasileira (1889-1930) Poder Legislativo fechado entre 15 de novembro de 1889 e 1° de junho de 1891',
       'Imagem Primeira República Brasileira (1889-1930) Poder Legislativo fechado entre 15 de novembro de 1889 e 1° de junho de 1891',
       'Início do mandato Primeira República Brasileira (1889-1930) Poder Legislativo fechado entre 15 de novembro de 1889 e 1° de junho de 1891',
       'Fim do mandato Primeira República Brasileira (1889-1930) Poder Legislativo fechado entre 15 de novembro de 1889 e 1° de junho de 1891',
       'Estado pelo qual foi Eleito Primeira República Brasileira (1889-1930) Poder Legislativo fechado entre 15 de novembro de 1889 e 1° de junho de 1891',
       'Partido Primeira República Brasileira (1889-1930) Poder Legislativo fechado entre 15 de novembro de 1889 e 1° de junho de 1891',
 

In [125]:
df.rename(columns={df.columns[3]: "DataInicial", 
                   df.columns[4]: "DataFinal",
                   df.columns[7]: "Partido"
                   }, inplace=True)

In [126]:
def custom_date_parser(date_str):
    months = {
        'janeiro': '01', 'fevereiro': '02', 'março': '03', 'abril': '04',
        'maio': '05', 'junho': '06', 'julho': '07', 'agosto': '08',
        'setembro': '09', 'outubro': '10', 'novembro': '11', 'dezembro': '12'
    }
    day, month_str, year = date_str.split(' de ')
    
    if '1' in day:
        day = '1'

    month = months[month_str]
    return f'{year}-{month}-{day}'

In [127]:
def extract_year(date_str):
    try:
        # Extrair a última palavra da string, que é o ano
        year = date_str.split()[-1]
        # Verificar se o ano é um número válido
        if year.isdigit():
            return int(year)
        else:
            return None
    except Exception as e:
        return None

In [128]:
df['AnoInicial'] = df['DataInicial'].apply(extract_year)
df['AnoFinal'] = df['DataFinal'].apply(extract_year)


In [129]:
df = df[["DataInicial", "DataFinal", "AnoInicial", "AnoFinal",  "Partido"]]
df

,DataInicial,DataFinal,AnoInicial,AnoFinal,Partido
0,18 de junho de 1891,31 de outubro de 1891,1891.0,1891.0,Partido Republicano Mineiro PRM
1,31 de outubro de 1891,18 de agosto de 1892,1891.0,1892.0,Partido Republicano Paulista PRP
2,18 de agosto de 1892,17 de maio de 1894,1892.0,1894.0,Partido Federalista[13] PF
3,17 de maio de 1894,19 de maio de 1896,1894.0,1896.0,Partido Republicano Federal PRF
4,19 de maio de 1896,9 de maio de 1899,1896.0,1899.0,Partido Republicano Baiano PRB
...,...,...,...,...,...
67,14 de julho de 2016,2 de fevereiro de 2017,2016.0,2017.0,Democratas DEM
68,2 de Fevereiro de 2017,1º de Fevereiro de 2019,2017.0,2019.0,Democratas DEM
69,2 de fevereiro de 2019,1° de fevereiro de 2021,2019.0,2021.0,Democratas DEM
70,2 de fevereiro de 2021,1º de fevereiro de 2023,2021.0,2023.0,Progressistas PP


In [130]:
df_filtered_anos = df[df['AnoFinal'] >=2013]

In [131]:
df_filtered_anos


,DataInicial,DataFinal,AnoInicial,AnoFinal,Partido
63,2 de fevereiro de 2011,4 de fevereiro de 2013,2011.0,2013.0,Partido dos Trabalhadores PT
64,4 de fevereiro de 2013,1º de fevereiro de 2015,2013.0,2015.0,Partido do Movimento Democrático Brasileiro PMDB
66,5 de maio de 2016,14 de julho de 2016,2016.0,2016.0,Partido Progressista PP
67,14 de julho de 2016,2 de fevereiro de 2017,2016.0,2017.0,Democratas DEM
68,2 de Fevereiro de 2017,1º de Fevereiro de 2019,2017.0,2019.0,Democratas DEM
69,2 de fevereiro de 2019,1° de fevereiro de 2021,2019.0,2021.0,Democratas DEM
70,2 de fevereiro de 2021,1º de fevereiro de 2023,2021.0,2023.0,Progressistas PP
71,2 de fevereiro de 2023,1º de fevereiro de 2025,2023.0,2025.0,Progressistas PP


TRansformando Anos em int e formatando Partido

In [132]:
df_filtered_anos.loc[:, "AnoInicial"] = df_filtered_anos["AnoInicial"].astype(int)
df_filtered_anos.loc[:, "AnoFinal"] = df_filtered_anos["AnoFinal"].astype(int)
df_filtered_anos.loc[:, "Partido"] = df_filtered_anos["Partido"].apply(lambda x: x.split()[-1])
print(df_filtered_anos)

               DataInicial                DataFinal  AnoInicial  AnoFinal  \
63  2 de fevereiro de 2011   4 de fevereiro de 2013      2011.0    2013.0   
64  4 de fevereiro de 2013  1º de fevereiro de 2015      2013.0    2015.0   
66       5 de maio de 2016      14 de julho de 2016      2016.0    2016.0   
67     14 de julho de 2016   2 de fevereiro de 2017      2016.0    2017.0   
68  2 de Fevereiro de 2017  1º de Fevereiro de 2019      2017.0    2019.0   
69  2 de fevereiro de 2019  1° de fevereiro de 2021      2019.0    2021.0   
70  2 de fevereiro de 2021  1º de fevereiro de 2023      2021.0    2023.0   
71  2 de fevereiro de 2023  1º de fevereiro de 2025      2023.0    2025.0   

   Partido  
63      PT  
64    PMDB  
66      PP  
67     DEM  
68     DEM  
69     DEM  
70      PP  
71      PP  


TRansformando em Datetime

In [133]:
df_filtered_anos.loc[:, 'DataFinal'] = df_filtered_anos['DataFinal'].str.replace('1º', '1', regex=True).str.replace('Fevereiro', 'fevereiro')
df_filtered_anos.loc[:, 'DataInicial'] = df_filtered_anos['DataInicial'].str.replace('Fevereiro', 'fevereiro')
df_filtered_anos.loc[:, 'DataInicial'] = df_filtered_anos['DataInicial'].apply(custom_date_parser)
df_filtered_anos.loc[:, 'DataFinal'] = df_filtered_anos['DataFinal'].apply(custom_date_parser)


In [134]:
print(df_filtered_anos)

   DataInicial  DataFinal  AnoInicial  AnoFinal Partido
63   2011-02-2  2013-02-4      2011.0    2013.0      PT
64   2013-02-4  2015-02-1      2013.0    2015.0    PMDB
66   2016-05-5  2016-07-1      2016.0    2016.0      PP
67   2016-07-1  2017-02-2      2016.0    2017.0     DEM
68   2017-02-2  2019-02-1      2017.0    2019.0     DEM
69   2019-02-2  2021-02-1      2019.0    2021.0     DEM
70   2021-02-2  2023-02-1      2021.0    2023.0      PP
71   2023-02-2  2025-02-1      2023.0    2025.0      PP


In [135]:
df_filtered_anos.loc[:, 'DataInicial'] = pd.to_datetime(df_filtered_anos['DataInicial'], format='%Y-%m-%d', errors='coerce').dt.date
df_filtered_anos.loc[:, 'DataFinal'] = pd.to_datetime(df_filtered_anos['DataFinal'], format='%Y-%m-%d', errors='coerce').dt.date


In [136]:
print(df_filtered_anos)

   DataInicial   DataFinal  AnoInicial  AnoFinal Partido
63  2011-02-02  2013-02-04      2011.0    2013.0      PT
64  2013-02-04  2015-02-01      2013.0    2015.0    PMDB
66  2016-05-05  2016-07-01      2016.0    2016.0      PP
67  2016-07-01  2017-02-02      2016.0    2017.0     DEM
68  2017-02-02  2019-02-01      2017.0    2019.0     DEM
69  2019-02-02  2021-02-01      2019.0    2021.0     DEM
70  2021-02-02  2023-02-01      2021.0    2023.0      PP
71  2023-02-02  2025-02-01      2023.0    2025.0      PP


In [137]:
#df_filtered_anos.to_csv('../data/presidentes_camara_deputados.csv', index=False)
#print('Arquivo salvo com sucesso!')